In [4]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier

In [5]:
def adaboost_step(X, y, sample_weights):
    weak_classifier = DecisionTreeClassifier(max_depth=1)
    weak_classifier.fit(X, y, sample_weight=sample_weights)

    predictions = weak_classifier.predict(X)

    err = np.sum(sample_weights * (predictions != y)) / np.sum(sample_weights)

    alpha = 0.5 * np.log((1 - err) / (err + 1e-10))

    sample_weights *= np.exp(-alpha * y * predictions)
    sample_weights /= np.sum(sample_weights)

    return weak_classifier, alpha, sample_weights

def adaboost_fit(X, y, n_estimators=50):
    n_samples = X.shape[0]
    sample_weights = np.ones(n_samples) / n_samples

    classifiers = []
    alphas = []

    for _ in range(n_estimators):
        classifier, alpha, sample_weights = adaboost_step(X, y, sample_weights)
        classifiers.append(classifier)
        alphas.append(alpha)

    return classifiers, alphas

def adaboost_predict(X, classifiers, alphas):
    final_predictions = np.zeros(X.shape[0])

    for classifier, alpha in zip(classifiers, alphas):
        predictions = classifier.predict(X)
        final_predictions += alpha * predictions

    return np.sign(final_predictions)

In [6]:
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=100, n_features=10, random_state=42)
y = np.where(y == 0, -1, 1) 

classifiers, alphas = adaboost_fit(X, y, n_estimators=50)

predictions = adaboost_predict(X, classifiers, alphas)

accuracy = np.mean(predictions == y)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 1.00
